In [ ]:
##draw normal background
from mmpose.apis import MMPoseInferencer
import json
import cv2
import numpy as np
import os
import math
from matplotlib import pyplot as plt
import subprocess
from PIL import Image, ImageOps
from IPython.display import clear_output
import shutil

####parameter
folder_all_path = r'F:\mmpose\same_color_background_r'
source_path = r'F:\mmpose\predictions'

file_all_list = os.listdir(folder_all_path) #此步驟是找到資料夾中有哪幾組資料並列出
inferencer = MMPoseInferencer('body26') #使用body26 model

for file_all_name in file_all_list: #迴圈遍尋檔案夾名稱
    folder_path = os.path.join(folder_all_path, file_all_name, 'processed_video_yolo_human_body_25') #找到該資料中欲解析的影片
    file_list = os.listdir(folder_path) #所有影片的檔名
    
    json_path = os.path.join(folder_all_path, file_all_name, 'json_file') #創造json的儲存路徑
    if not os.path.exists(json_path):# 確保目標資料夾存在，如果不存在，則創建它
        os.makedirs(json_path)
    
    new_mmpose_processed_path = os.path.join(folder_all_path, file_all_name, 'mmpose_processed_video') #創造用.json檔貼回處理過影片路徑
    if not os.path.exists(new_mmpose_processed_path):# 確保目標資料夾存在，如果不存在，則創建它
        os.makedirs(new_mmpose_processed_path)

    origin_path = os.path.join(folder_all_path, file_all_name, 'same_color_background')
    
    new_mmpose_path = os.path.join(folder_all_path, file_all_name, 'mmpose_same_color') #創造用.json檔貼回原影片路徑
    if not os.path.exists(new_mmpose_path):# 確保目標資料夾存在，如果不存在，則創建它
        os.makedirs(new_mmpose_path)

    list_new = [new_mmpose_processed_path, new_mmpose_path]
    list_old = [folder_path, origin_path]
    
    for file_name in file_list: #迴圈逐個開啟檔案
        name_j = file_name.replace('.mp4','.json') #將json檔的名稱設為與mp4相同以比對
        if not os.path.exists(os.path.join(json_path,name_j)):#若目的地已經有相同結果，則跳過避免重複計算
            if file_name.endswith('.mp4'): # 檢查檔案是否為MP4檔案
                file_path = os.path.join(folder_path, file_name) #組合完整的檔案路徑
                result_generator = inferencer(file_path, pred_out_dir='predictions')#運算json資料並儲存到mmpose的predictions
                results = [result for result in result_generator]
    
    for file_name in os.listdir(source_path): #遍歷predictions資料夾中的 JSON 檔案，進行複製
        if file_name.endswith('.json'):
            source = os.path.join(source_path, file_name)
            target = os.path.join(json_path, file_name)
            # 複製檔案
            shutil.copy2(source, target)
            # 刪除原始資料夾中的檔案
            os.remove(source)
    
    name_j = os.listdir(json_path)

    for j in range(2):
        count_json = -1
        for index, file in enumerate(os.listdir(list_old[j]), start=1):
            count_json = count_json + 1
            video_full_path = os.path.join(list_old[j], file)
            output = os.path.join(list_new[j],file)
            if not os.path.exists(os.path.join(list_new[j], file)):
                cap = cv2.VideoCapture(video_full_path)
                fps = cap.get(cv2.CAP_PROP_FPS)
                height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                line = [[0,1],[1,3],[0,2],[2,4],[0,17],[0,18],[18,5],[5,7],[7,9],[18,6],[6,8],[8,10],[18,19],[19,11],[11,13],[13,15],[15,24],[15,20],[15,22],[19,12],[12,14],[14,16],[16,25],[16,21],[16,23]]
                fps = cap.get(cv2.CAP_PROP_FPS)
                height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                out = cv2.VideoWriter(output, fourcc, fps, (width, height))
                count_frame = 0
                while True:
                    ret, frame = cap.read()
                    if not ret:
                             break
                    f = open(os.path.join(json_path, name_j[count_json]))
                    temp = json.load(f) #載入同檔名的json file
                    #第一個0代表第幾幀，第二個0代表畫面中的第幾+1位辨識體，第三個0代表第幾個節點
                    keypoints = temp[count_frame]["instances"][0]['keypoints'] #呼叫該檔案中的keypoints列表
                    keypoint_scores = temp[count_frame]["instances"][0]['keypoint_scores'] #呼叫該檔案中的keypoint_scores列表
                    count = 0
                    for i in range(26):
                        p = keypoint_scores[count]*255 #隨score顏色進行變換
                        #若keypoint score太低，則標示出來
                        if keypoint_scores[count]>=0.3 and keypoint_scores[count]<0.5:
                            cv2.circle(frame,(int(keypoints[count][0]), int(keypoints[count][1])), 3, (255-p, p, 0), 3)#frame, (x,y), radius, color, thickness
                            font = cv2.FONT_HERSHEY_SIMPLEX # font
                            org = (int(keypoints[count][0])+3, int(keypoints[count][1])) # 偏移
                            fontScale = 0.5 # fontScale
                            color = (255, 255, 255) # Blue color in BGR
                            thickness = 1 # Line thickness of 2 px 
                            # Using cv2.putText() method 
                            image = cv2.putText(frame, str(int(keypoint_scores[count]*100)), org, font,  
                                                   fontScale, color, thickness, cv2.LINE_AA) 
                        elif keypoint_scores[count]>=0.5:
                            cv2.circle(frame,(int(keypoints[count][0]), int(keypoints[count][1])), 2, (0, 255, p), 3)
                        count = count+1
                            
                    for i in range(25):
                        if keypoint_scores[line[i][0]]>0.3 and keypoint_scores[line[i][1]]>0.3: 
                            
                            cv2.line(frame, (int(keypoints[line[i][0]][0]), int(keypoints[line[i][0]][1])), (int(keypoints[line[i][1]][0]), int(keypoints[line[i][1]][1])), (0, 0, 255), 1)
                    out.write(frame)
                    count_frame = count_frame+1
                print(file+' finished')
                cap.release()
                out.release()
        clear_output(wait=False)

